<a href="https://colab.research.google.com/github/MickDobbsKildavin2/DAV/blob/main/Prog4BigDataPySparkColab_GettingStarted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Run Once
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://www-us.apache.org/dist/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz

# check if the file is unziped 
!ls

!pip install -q findspark
#Run Once
import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"
import findspark
findspark.init()

sample_data  spark-3.0.2-bin-hadoop2.7	spark-3.0.2-bin-hadoop2.7.tgz


In [ ]:
#!rm spark*3.2.0*.*
#!ls


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

In [ ]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data" -O sample_data/iris.data

--2022-03-03 18:50:03--  https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551 (4.4K) [application/x-httpd-php]
Saving to: ‘sample_data/iris.data’

sample_data/iris.da 100%[===================>]   4.44K  --.-KB/s    in 0s      

2022-03-03 18:50:03 (144 MB/s) - ‘sample_data/iris.data’ saved [4551/4551]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
df = spark.read.csv('sample_data/iris.data', header=False, sep=",", inferSchema=True)
df.printSchema()

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: string (nullable = true)



In [ ]:
from pyspark.sql.types import *
df.columns

['_c0', '_c1', '_c2', '_c3', '_c4']

Import with pre-defined schema

In [ ]:
labels = [
     ('SepalLength',DoubleType()),
     ('SepalWidth',DoubleType()),
     ('PetalLength',DoubleType()),
     ('PetalWidth',DoubleType()),
     ('Class', StringType()),
]

In [ ]:
# Create the schema 
schema = StructType([StructField (x[0], x[1], True) for x in labels])
schema

StructType(List(StructField(SepalLength,DoubleType,true),StructField(SepalWidth,DoubleType,true),StructField(PetalLength,DoubleType,true),StructField(PetalWidth,DoubleType,true),StructField(Class,StringType,true)))

In [ ]:
df = spark.read.csv('sample_data/iris.data', header=True, sep=",", schema=schema)
df.printSchema()
# Check schema 

root
 |-- SepalLength: double (nullable = true)
 |-- SepalWidth: double (nullable = true)
 |-- PetalLength: double (nullable = true)
 |-- PetalWidth: double (nullable = true)
 |-- Class: string (nullable = true)



In [ ]:
df.columns

['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Class']

In [ ]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+--------------+
|summary|       SepalLength|        SepalWidth|       PetalLength|        PetalWidth|         Class|
+-------+------------------+------------------+------------------+------------------+--------------+
|  count|               149|               149|               149|               149|           149|
|   mean| 5.848322147651008| 3.051006711409397|3.7744966442953043|1.2053691275167793|          null|
| stddev|0.8285940572656166|0.4334988777167475|1.7596511617753412|0.7612920413899604|          null|
|    min|               4.3|               2.0|               1.0|               0.1|   Iris-setosa|
|    max|               7.9|               4.4|               6.9|               2.5|Iris-virginica|
+-------+------------------+------------------+------------------+------------------+--------------+



In [ ]:
df.filter("SepalLength>5").show(5)

+-----------+----------+-----------+----------+-----------+
|SepalLength|SepalWidth|PetalLength|PetalWidth|      Class|
+-----------+----------+-----------+----------+-----------+
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|
|        5.4|       3.7|        1.5|       0.2|Iris-setosa|
|        5.8|       4.0|        1.2|       0.2|Iris-setosa|
|        5.7|       4.4|        1.5|       0.4|Iris-setosa|
|        5.4|       3.9|        1.3|       0.4|Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 5 rows



In [ ]:
df.filter("SepalLength>5").select('SepalLength','Class').show(5)

+-----------+-----------+
|SepalLength|      Class|
+-----------+-----------+
|        5.4|Iris-setosa|
|        5.4|Iris-setosa|
|        5.8|Iris-setosa|
|        5.7|Iris-setosa|
|        5.4|Iris-setosa|
+-----------+-----------+
only showing top 5 rows



In [ ]:
df.filter("SepalLength<5").select(['SepalWidth','Class']).show(5)

+----------+-----------+
|SepalWidth|      Class|
+----------+-----------+
|       3.0|Iris-setosa|
|       3.2|Iris-setosa|
|       3.1|Iris-setosa|
|       3.4|Iris-setosa|
|       2.9|Iris-setosa|
+----------+-----------+
only showing top 5 rows



In [ ]:
df.filter( (df["SepalWidth"] >4) & (df['SepalLength'] >3) ).show()

+-----------+----------+-----------+----------+-----------+
|SepalLength|SepalWidth|PetalLength|PetalWidth|      Class|
+-----------+----------+-----------+----------+-----------+
|        5.7|       4.4|        1.5|       0.4|Iris-setosa|
|        5.2|       4.1|        1.5|       0.1|Iris-setosa|
|        5.5|       4.2|        1.4|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+



In [ ]:
df.filter(df["SepalWidth"] == 4).show()

+-----------+----------+-----------+----------+-----------+
|SepalLength|SepalWidth|PetalLength|PetalWidth|      Class|
+-----------+----------+-----------+----------+-----------+
|        5.8|       4.0|        1.2|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+



In [ ]:
df.groupBy("Class").mean().show()

+---------------+-----------------+------------------+------------------+------------------+
|          Class| avg(SepalLength)|   avg(SepalWidth)|  avg(PetalLength)|   avg(PetalWidth)|
+---------------+-----------------+------------------+------------------+------------------+
| Iris-virginica|6.587999999999998|2.9739999999999998|             5.552|             2.026|
|    Iris-setosa|5.004081632653061| 3.416326530612245|1.4653061224489796|0.2448979591836734|
|Iris-versicolor|            5.936|2.7700000000000005|              4.26|1.3259999999999998|
+---------------+-----------------+------------------+------------------+------------------+



In [ ]:
df.groupBy("Class").count().show()

+---------------+-----+
|          Class|count|
+---------------+-----+
| Iris-virginica|   50|
|    Iris-setosa|   49|
|Iris-versicolor|   50|
+---------------+-----+



In [ ]:
df.groupBy("Class").max().show()

+---------------+----------------+---------------+----------------+---------------+
|          Class|max(SepalLength)|max(SepalWidth)|max(PetalLength)|max(PetalWidth)|
+---------------+----------------+---------------+----------------+---------------+
| Iris-virginica|             7.9|            3.8|             6.9|            2.5|
|    Iris-setosa|             5.8|            4.4|             1.9|            0.6|
|Iris-versicolor|             7.0|            3.4|             5.1|            1.8|
+---------------+----------------+---------------+----------------+---------------+



In [ ]:
df.groupBy("Class").min().show()

+---------------+----------------+---------------+----------------+---------------+
|          Class|min(SepalLength)|min(SepalWidth)|min(PetalLength)|min(PetalWidth)|
+---------------+----------------+---------------+----------------+---------------+
| Iris-virginica|             4.9|            2.2|             4.5|            1.4|
|    Iris-setosa|             4.3|            2.3|             1.0|            0.1|
|Iris-versicolor|             4.9|            2.0|             3.0|            1.0|
+---------------+----------------+---------------+----------------+---------------+



In [ ]:
df.groupBy("Class").sum().show()

+---------------+------------------+------------------+------------------+------------------+
|          Class|  sum(SepalLength)|   sum(SepalWidth)|  sum(PetalLength)|   sum(PetalWidth)|
+---------------+------------------+------------------+------------------+------------------+
| Iris-virginica| 329.3999999999999|             148.7|277.59999999999997|101.29999999999998|
|    Iris-setosa|245.19999999999996|             167.4|              71.8|11.999999999999996|
|Iris-versicolor|             296.8|138.50000000000003|212.99999999999997|              66.3|
+---------------+------------------+------------------+------------------+------------------+



In [ ]:
# Avg SepalWidth 
df.agg({'SepalWidth':'avg'}).show()

+-----------------+
|  avg(SepalWidth)|
+-----------------+
|3.051006711409397|
+-----------------+



In [ ]:
from pyspark.sql.functions import countDistinct, avg, stddev
from pyspark.sql.functions import format_number
sales_std = df.select(stddev("SepalWidth").alias('std'))
sales_std.show()
sales_std.select(format_number('std',2)).show()

+------------------+
|               std|
+------------------+
|0.4334988777167475|
+------------------+

+---------------------+
|format_number(std, 2)|
+---------------------+
|                 0.43|
+---------------------+



In [ ]:
grouped = df.groupBy("Class")
grouped.agg({"SepalWidth":'avg'}).show()

+---------------+------------------+
|          Class|   avg(SepalWidth)|
+---------------+------------------+
| Iris-virginica|2.9739999999999998|
|    Iris-setosa| 3.416326530612245|
|Iris-versicolor|2.7700000000000005|
+---------------+------------------+



In [ ]:
from pyspark.sql.functions import countDistinct, avg, stddev
df.select(countDistinct("SepalWidth")).show()

+--------------------------+
|count(DISTINCT SepalWidth)|
+--------------------------+
|                        23|
+--------------------------+



In [ ]:
df.select(countDistinct("SepalWidth").alias("Distinct SepalWidths")).show()

+--------------------+
|Distinct SepalWidths|
+--------------------+
|                  23|
+--------------------+



In [ ]:
df.select(avg('SepalWidth')).show()

+-----------------+
|  avg(SepalWidth)|
+-----------------+
|3.051006711409397|
+-----------------+



In [ ]:
df.select(stddev("SepalWidth")).show()

+-----------------------+
|stddev_samp(SepalWidth)|
+-----------------------+
|     0.4334988777167475|
+-----------------------+



In [ ]:
df.orderBy("SepalWidth").show(5)

+-----------+----------+-----------+----------+---------------+
|SepalLength|SepalWidth|PetalLength|PetalWidth|          Class|
+-----------+----------+-----------+----------+---------------+
|        5.0|       2.0|        3.5|       1.0|Iris-versicolor|
|        6.2|       2.2|        4.5|       1.5|Iris-versicolor|
|        6.0|       2.2|        5.0|       1.5| Iris-virginica|
|        6.0|       2.2|        4.0|       1.0|Iris-versicolor|
|        4.5|       2.3|        1.3|       0.3|    Iris-setosa|
+-----------+----------+-----------+----------+---------------+
only showing top 5 rows



In [ ]:
df.orderBy(df["SepalWidth"].desc()).show()

+-----------+----------+-----------+----------+--------------+
|SepalLength|SepalWidth|PetalLength|PetalWidth|         Class|
+-----------+----------+-----------+----------+--------------+
|        5.7|       4.4|        1.5|       0.4|   Iris-setosa|
|        5.5|       4.2|        1.4|       0.2|   Iris-setosa|
|        5.2|       4.1|        1.5|       0.1|   Iris-setosa|
|        5.8|       4.0|        1.2|       0.2|   Iris-setosa|
|        5.4|       3.9|        1.7|       0.4|   Iris-setosa|
|        5.4|       3.9|        1.3|       0.4|   Iris-setosa|
|        7.9|       3.8|        6.4|       2.0|Iris-virginica|
|        5.1|       3.8|        1.5|       0.3|   Iris-setosa|
|        7.7|       3.8|        6.7|       2.2|Iris-virginica|
|        5.7|       3.8|        1.7|       0.3|   Iris-setosa|
|        5.1|       3.8|        1.9|       0.4|   Iris-setosa|
|        5.1|       3.8|        1.6|       0.2|   Iris-setosa|
|        5.1|       3.7|        1.5|       0.4|   Iris-